In [1]:
import torch
import torch.utils.data
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import numpy as np
import h5py
from data_utils import get_data

/home/zeyudesk/anaconda3/envs/ECE232project/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Load data from all .mat files, combine them, eliminate EOG signals, shuffle and 
# seperate training data, validation data and testing data.
# Also do mean subtraction on x.

data = get_data('project_datasets',num_validation=100, num_test=100, subtract_mean=False, transpose=True)
for k in data.keys():
    print('{}: {} '.format(k, data[k].shape))

X_train: (2358, 1000, 22) 
y_train: (2358,) 
X_val: (100, 1000, 22) 
y_val: (100,) 
X_test: (100, 1000, 22) 
y_test: (100,) 


In [3]:
print(data.get('X_test'))

[[[ -7.12890625  -8.10546875  -8.0078125  ...  -3.56445312  -3.27148438
    -2.49023438]
  [ -3.3203125   -0.87890625  -3.515625   ...   3.22265625   2.29492188
     5.71289062]
  [ -2.63671875  -1.5625      -2.58789062 ...   0.87890625   2.00195312
     4.98046875]
  ...
  [-21.24023438 -17.578125   -14.50195312 ...   1.22070312   4.6875
     5.2734375 ]
  [-24.90234375 -13.76953125 -16.65039062 ...   2.68554688   6.49414062
     8.83789062]
  [-23.87695312 -17.04101562 -15.96679688 ...   4.93164062   8.15429688
    12.35351562]]

 [[ -9.32617188 -16.11328125 -14.35546875 ... -14.40429688  -7.76367188
    -9.32617188]
  [-11.37695312 -19.48242188 -17.1875     ... -14.01367188  -5.51757812
    -6.25      ]
  [ -3.3203125  -11.328125    -8.59375    ...  -4.19921875   3.17382812
     3.07617188]
  ...
  [  0.53710938  -1.7578125   -2.49023438 ...   5.37109375   7.32421875
     7.37304688]
  [  1.26953125  -3.46679688  -2.09960938 ...   4.19921875   6.78710938
     5.90820312]
  [  2.9296

In [4]:
# class flatten to connect to FC layer
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H = x.size() # read in N, C, H
        return x.view(N, -1)

In [5]:
# turn x and y into torch type tensor
dtype = torch.FloatTensor

X_train = Variable(torch.Tensor(data.get('X_train'))).type(dtype)
y_train = Variable(torch.Tensor(data.get('y_train'))).type(dtype)
X_val = Variable(torch.Tensor(data.get('X_val'))).type(dtype)
y_val = Variable(torch.Tensor(data.get('y_val'))).type(dtype)
X_test = Variable(torch.Tensor(data.get('X_test'))).type(dtype)
y_test = Variable(torch.Tensor(data.get('y_test'))).type(dtype)

In [105]:
# class for RNN model

class RNN(nn.Module):
    # input_size: number of EGG channels
    # output_size: number of classes
    # hidden_size: size of hidden layers in RNN module
    # num_layers: number of hidden layers in RNN module
    # dropout: non-zero value means applying dropout to hidden layers
    def __init__(self, input_size, output_size, hidden_size=10, num_layers=1, dropout=0):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.output = nn.Linear(hidden_size, output_size)
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self, x, y, hidden_init, T_length=1):
        N, _, _ = x.size()
        out_rnn, state_rnn = self.rnn(x, hidden_init)
        score = self.output(out_rnn)
        
        # calculate the total loss at time interval of T_length
        loss = 0
        for t in np.arange(T_length):
            yy = score[:,-(t+1),:].squeeze(1)
            loss += self.loss_fn(yy,y)
        # score[N,S,C]:
        # N: batch_size(number of time series), S: length of time series, C: number of classes
        return score, loss, state_rnn
    

In [107]:
# decare a RNN model

N, S, input_size = X_train.size()
output_size = 4
hidden_size = 40
num_layers = 1
dropout = 0

model = RNN(input_size, output_size, hidden_size, num_layers, dropout)
model.type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-2)


In [114]:
# train on several time intervals with all data sequence
epoch = 100
T_batch_size = int(S/epoch)

for t in range(epoch):    
    if t == 0:state_rnn = Variable(torch.zeros(num_layers, N, hidden_size))
        
    # calculate loss
    X_train_sub = X_train[:,t*T_batch_size:(t+1)*T_batch_size,:]    
    score, loss, state_rnn = model.forward(X_train_sub, y_train.type(torch.LongTensor), state_rnn, T_batch_size)
    #print(loss.size(), score.size(), state_rnn.size())
    
    # backpropagation
    model.zero_grad()
    loss.backward(retain_graph=True)
    
    # update parameters
    optimizer.step()
    
    print(t,loss)

0 Variable containing:
 13.9013
[torch.FloatTensor of size 1]

1 Variable containing:
 13.9152
[torch.FloatTensor of size 1]

2 Variable containing:
 13.8985
[torch.FloatTensor of size 1]

3 Variable containing:
 13.9061
[torch.FloatTensor of size 1]

4 Variable containing:
 13.8844
[torch.FloatTensor of size 1]

5 Variable containing:
 13.8800
[torch.FloatTensor of size 1]

6 Variable containing:
 13.8798
[torch.FloatTensor of size 1]

7 Variable containing:
 13.8973
[torch.FloatTensor of size 1]

8 Variable containing:
 13.8992
[torch.FloatTensor of size 1]

9 Variable containing:
 13.9032
[torch.FloatTensor of size 1]

10 Variable containing:
 13.8969
[torch.FloatTensor of size 1]

11 Variable containing:
 13.8972
[torch.FloatTensor of size 1]

12 Variable containing:
 13.8847
[torch.FloatTensor of size 1]

13 Variable containing:
 13.8809
[torch.FloatTensor of size 1]

14 Variable containing:
 13.8851
[torch.FloatTensor of size 1]

15 Variable containing:
 13.8915
[torch.FloatTenso

In [16]:
np.set_printoptions(threshold=np.nan)
y_temp = y_pred.data.numpy()
#print(y_temp)
np.isnan(np.sum(y_temp))


False